In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from BICS_ABM import BICS_ABM, VaccineRule
import time
from joblib import Parallel, delayed
from scipy.stats.qmc import LatinHypercube
from scipy.stats import norm, uniform, randint

import collections
import re

import datetime
import pickle
import dill

from scipy.signal import find_peaks

# Analysis for new idea: Vaccine timing and outbreak cycles

Begin by developing treatment combinations

In [ ]:
t_reinfection_vec = [180*24, 270*24, 360*24]
print(t_reinfection_vec)
t_booster_vec = np.cumsum([0, 31,28,31,30,31,30,31,31,30,31,30])
print(t_booster_vec)
ve_boost_vec = [.25, .50, .75]
print(ve_boost_vec)
ve_uptake_vec = ve_boost_vec

treatment_params = []
for r in t_reinfection_vec:
    for b in t_booster_vec:
        for ve in ve_boost_vec:
            for vu in ve_uptake_vec:
                treatment_params.append({'T_REINFECTION': r,
                                         'BOOSTER_DAY': b,
                                         'VEBOOST': ve,
                                         'VU': vu})
                


In [ ]:
len(treatment_params)

In [ ]:
t = time.time()
import_cases = [1 if x%7 == 0 else 0 for x in range(365)]

def multi_fn(params):
    """ Wrapper function for multiprocessing
    
    """
    

    result = BICS_ABM( 
            N_HH = 1000,
            # SEED = 4949, 
            RHO = 1,
            ALPHA = 0.25,
            BETA0 = 0.05,
            BETA1 = 0.5,
            INDEX_CASES = 1,
            IMPORT_CASES_VEC = import_cases,
            # T0 = 60,
            ISOLATION_MULTIPLIER = 0.1, 
            N_VAX_DAILY = 100,
            T_REINFECTION = params['T_REINFECTION'],
            VEBOOST = params['VEBOOST'],
            vax_rules = [
                VaccineRule(general = True, hesitancy = params['VU']) #,
            ],
            MAX_DAYS=365*10,
            BOOSTER_DAY = params['BOOSTER_DAY'],
            silent = True)
    
    fname = "sims/2023_13_2/"
    for k, v in params.items():
        fname += k + "_" + str(v) + "__"
    fname += ".pickle"
    outfile = open(fname,'wb')
    pickle.dump(result, outfile)
    outfile.close()
    
    return 0 # result
    
    
result_vec = Parallel(n_jobs=8, verbose = 10)(delayed(multi_fn)(v) for v in treatment_params)
    
    
t = time.time() - t

print(len(treatment_params), "iterations took", t, "seconds")


In [ ]:
import os
flist = os.listdir("sims/2023_13_2/")

sims = []
for i,f in enumerate(flist):
    
    try:
        p = open("sims/2023_13_2/" + f, "rb")
        sims.append(pickle.load(p))
        p.close()
        
    except:
        print("Failed at", i)

In [ ]:
# Functions for analysis
from scipy.signal import find_peaks

def process_sim(sim):
    
    vu = np.mean(sim._pop[:,[i for i,x in enumerate(sim._colnames) if x == "vaccine_priority" ][0]] == 0)
    
    ret = {'T_REINFECTION': sim._params.T_REINFECTION,
              'BOOSTER_DAY': sim._params.BOOSTER_DAY,
              'VEBOOST': sim._params.VEBOOST,
              'VU': vu}
    N = sim.S[0] + sim.E[0]
    threshold = N * 0.01
    threshold= 2
    x = np.add.reduceat(sim.Cc, np.arange(0, len(sim.Cc), 24))
    peaks, properties = find_peaks(x, height=threshold, distance=30, width=1, rel_height=1)
    
    count = []
    for i in range(len(properties["left_ips"])):
        left_bound = round(properties["left_ips"][i])
        right_bound = round(properties["right_ips"][i])
        count.append(sum(x[left_bound:right_bound]))
            
    ret["num_outbreaks"] = len(peaks)
    ret["mean_time_bw_outbreaks"] = np.mean(np.diff(peaks))
    ret["avg_outbreak_size"] = np.mean(count)
    
    return ret
    

In [ ]:
processed_sims = [process_sim(x) for x in sims]

In [ ]:
processed_sims = pd.DataFrame(processed_sims)

In [ ]:
processed_sims

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize= (10,5))
ax[0].plot(processed_sims.T_REINFECTION/24, processed_sims.num_outbreaks, ".", )
ax[0].set_title("Number of Outbreaks")
ax[0].set_xlabel("Time until reinfection (days)")
ax[1].plot(processed_sims.T_REINFECTION/24, processed_sims.mean_time_bw_outbreaks, ".")
ax[1].set_title("Avg. Time Between Outbreaks (days)")
ax[1].set_xlabel("Time until reinfection (days)")
ax[2].plot(processed_sims.T_REINFECTION/24, processed_sims.avg_outbreak_size, ".")
ax[2].set_title("Avg. Outbreak Size")
ax[2].set_xlabel("Time until reinfection (days)")

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize= (10,5))
ax[0].plot(processed_sims.BOOSTER_DAY, processed_sims.num_outbreaks, ".", )
ax[0].set_title("Number of Outbreaks")
ax[0].set_xlabel("Booster Day")
ax[1].plot(processed_sims.BOOSTER_DAY, processed_sims.mean_time_bw_outbreaks, ".")
ax[1].set_title("Avg. Time Between Outbreaks (days)")
ax[1].set_xlabel("Booster Day")
ax[2].plot(processed_sims.BOOSTER_DAY, processed_sims.avg_outbreak_size, ".")
ax[2].set_title("Avg. Outbreak Size")
ax[2].set_xlabel("Booster Day")

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize= (10,5))
ax[0].plot(processed_sims.VEBOOST, processed_sims.num_outbreaks, ".", )
ax[0].set_title("Number of Outbreaks")
ax[0].set_xlabel("Booster Efficacy")
ax[1].plot(processed_sims.VEBOOST, processed_sims.mean_time_bw_outbreaks, ".")
ax[1].set_title("Avg. Time Between Outbreaks (days)")
ax[1].set_xlabel("Booster Efficacy")
ax[2].plot(processed_sims.VEBOOST, processed_sims.avg_outbreak_size, ".")
ax[2].set_title("Avg. Outbreak Size")
ax[2].set_xlabel("Booster Efficacy")

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize= (10,5))
ax[0].plot(processed_sims.VU, processed_sims.num_outbreaks, ".", )
ax[0].set_title("Number of Outbreaks")
ax[0].set_xlabel("Vaccine Uptake Rate")
ax[1].plot(processed_sims.VU, processed_sims.mean_time_bw_outbreaks, ".")
ax[1].set_title("Avg. Time Between Outbreaks (days)")
ax[1].set_xlabel("Vaccine Uptake Rate")
ax[2].plot(processed_sims.VU, processed_sims.avg_outbreak_size, ".")
ax[2].set_title("Avg. Outbreak Size")
ax[2].set_xlabel("Vaccine Uptake Rate")